<a href="https://colab.research.google.com/github/arielmagbanua/image-classifiers/blob/main/mask-on-mask-off/moo_20210313-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import zipfile
import pathlib
import PIL
import numpy as np
from google.colab import files
from google.colab import drive
import time
import random

!pip install tensorflowjs

# mount the google drive
drive.mount('/content/drive')

     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 112kB 24.7MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Extract and Prepare Dataset

1.) Mount your Google Drive

2.) Designate a directory where you want to extract your dataset (E.g. `/content/mask-on-mask-off`).

In [2]:
# The path where to extract the dataset
ARCHIVED_DATASET_PATH = '/content/drive/MyDrive/mlds/dataset/mask_on_off.zip'
COLAB_DATA_SET_PATH = '/content/sample_data/mask-on-mask-off'

# clean up directory
!rm -R $COLAB_DATA_SET_PATH

zip_ref = zipfile.ZipFile(ARCHIVED_DATASET_PATH, 'r')
zip_ref.extractall(COLAB_DATA_SET_PATH)
zip_ref.close()

rm: cannot remove '/content/sample_data/mask-on-mask-off': No such file or directory


# Constants / Parameters

In [3]:
DESIRED_ACCURACY = 0.99

dataset_path = os.path.join(COLAB_DATA_SET_PATH)
print(dataset_path)

IMAGE_WIDTH = 512
IMAGE_HEIGHT = 512

/content/sample_data/mask-on-mask-off


# Callbacks and Early Stopping

In [4]:
class DesiredAccuracyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > DESIRED_ACCURACY:
      self.model.stop_training = True

desired_accuract_callback = DesiredAccuracyCallback()

# stop when loss is not decreasing
# loss_monitor_callback = tf.keras.callbacks.EarlyStopping(
#     monitor='loss',
#     mode='min',
#     patience=10
# )

# stop when accuracy is not inceasing
# accuracy_monitor_callback = tf.keras.callbacks.EarlyStopping(
#     monitor='accuracy',
#     mode='max',
#     patience=5
# )

# create the callbacks
callbacks = [desired_accuract_callback]

# Build the Model

In [11]:
model = tf.keras.models.Sequential([
  # input layer
  # first convolution
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),

  # second convolution
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  # third convolution
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  # fourth convolution
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  # fifth convolution
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  # Flatten the results to feed into a DNN
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(.5),
  tf.keras.layers.Dense(256, activation='relu'),

  # Only 1 output neuron.
  # It will contain a value from 0-1 where 0 for 1 class ('mask on') and 1 for the other ('mask off')
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 510, 510, 128)     3584      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 255, 255, 128)     0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 253, 253, 64)      73792     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 60, 60, 32)       

# Compile the Model

In [12]:
model.compile(
  loss='binary_crossentropy',
  optimizer = Adam(learning_rate=0.0003),
  metrics=['accuracy']
)

# Prepare Dataset with Data Generator

In [13]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
  rescale=1./255,
  shear_range=0.2,
  zoom_range=0.2,
  height_shift_range=0.2,
  rotation_range=30,
  horizontal_flip=True,
  validation_split=0.2
)

# training data generator
train_generator = train_datagen.flow_from_directory(
  dataset_path,
  target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
  batch_size=32,
  class_mode='binary',
  subset='training'
)

# validation data generator
validation_generator = train_datagen.flow_from_directory(
  dataset_path,
  target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
  batch_size=32,
  class_mode='binary',
  subset='validation'
)

Found 5056 images belonging to 2 classes.
Found 1264 images belonging to 2 classes.


# Train / Fit

In [ ]:
history = model.fit(
  train_generator,
  validation_data=validation_generator,
  steps_per_epoch=16,
  validation_steps=16,
  epochs=150,
  verbose=1,
  callbacks=callbacks
)

Epoch 1/150
16/16 [==============================] - 64s 4s/step - loss: 0.6939 - accuracy: 0.4807 - val_loss: 0.6927 - val_accuracy: 0.5020
Epoch 2/150
16/16 [==============================] - 61s 4s/step - loss: 0.6922 - accuracy: 0.5052 - val_loss: 0.6898 - val_accuracy: 0.6992
Epoch 3/150
16/16 [==============================] - 61s 4s/step - loss: 0.6884 - accuracy: 0.5954 - val_loss: 0.6792 - val_accuracy: 0.7461
Epoch 4/150
16/16 [==============================] - 61s 4s/step - loss: 0.6770 - accuracy: 0.5845 - val_loss: 0.6712 - val_accuracy: 0.4961
Epoch 5/150
16/16 [==============================] - 61s 4s/step - loss: 0.6280 - accuracy: 0.6825 - val_loss: 0.5250 - val_accuracy: 0.8145
Epoch 6/150
16/16 [==============================] - 61s 4s/step - loss: 0.5286 - accuracy: 0.7451 - val_loss: 0.4649 - val_accuracy: 0.8281
Epoch 7/150
16/16 [==============================] - 61s 4s/step - loss: 0.4542 - accuracy: 0.8269 - val_loss: 0.4046 - val_accuracy: 0.8438
Epoch 8/150
1

# Plot the Training History

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

#Evaluate the Model

In [ ]:
def infer(class_names, model, x):
  
  prediction = model.predict(x)
  result_index = np.argmax(prediction)
  probability = prediction[0][result_index]
  prediction_class_index = round(prediction[0][result_index])

  return probability, class_names[prediction_class_index]

In [ ]:
COLAB_EVAL_DATASET_PATH = '/content/drive/MyDrive/mlds/dataset/evaluation_data_set'
data_dir = pathlib.Path(COLAB_EVAL_DATASET_PATH)

eval_ds = tf.keras.preprocessing.image_dataset_from_directory(
  COLAB_EVAL_DATASET_PATH,
  seed=123,
  image_size=(IMAGE_WIDTH, IMAGE_HEIGHT)
)

class_names = eval_ds.class_names

# ma_onsk on inference example
samples = 9
mask_on_image_paths = list(data_dir.glob('mask_on/*.jpg'))
mask_off_image_paths = list(data_dir.glob('mask_off/*.jpg'))
image_paths = mask_on_image_paths + mask_off_image_paths
random.shuffle(image_paths)

paths_sample = random.sample(image_paths, samples)

plt.figure(figsize=(12, 12))

for i in range(samples):
  image = tf.keras.preprocessing.image.load_img(paths_sample[i])
  image = image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
  image = tf.keras.preprocessing.image.img_to_array(image)

  plt.subplot(3, 3, i + 1)
  plt.subplots_adjust(wspace=0.2, hspace=0.5)
  plt.imshow(image.astype("uint8"))
  # plt.axis("off")

  image_input = tf.expand_dims(image, axis=0)
  probability, class_name = infer(class_names, model, image_input)
  plt.title('prediction = {}\nclass = {}'.format(probability, class_name))

# Save the Model

In [ ]:
models_dir_path = '/content/models'
model_path = models_dir_path + '/moo.h5'

model.save(model_path)

# Convert the Model to Javascript

In [6]:
!tensorflowjs_converter --input_format keras $model_path $models_dir_path

zip_store_path = '/content/drive/MyDrive/ml_ai/moo_models_{}.zip'.format(int(time.time()))

!zip -r $zip_store_path $models_dir_path

2021-03-13 12:51:18.890641: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Traceback (most recent call last):
  File "/usr/local/bin/tensorflowjs_converter", line 8, in <module>
    sys.exit(pip_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorflowjs/converters/converter.py", line 813, in pip_main
    main([' '.join(sys.argv[1:])])
  File "/usr/local/lib/python3.7/dist-packages/tensorflowjs/converters/converter.py", line 817, in main
    convert(argv[0].split(' '))
  File "/usr/local/lib/python3.7/dist-packages/tensorflowjs/converters/converter.py", line 749, in convert
    'Missing input_path argument. For usage, use the --help flag.')
ValueError: Missing input_path argument. For usage, use the --help flag.

zip error: Invalid command arguments (cannot write zip file to terminal)


# Download All Models

In [ ]:
files.download(zip_store_path) 